## Trial-runs for model evalaution

In [48]:
import os 

In [49]:
os.chdir(r"C:\Users\Junior\OneDrive\Desktop\Heart-Attack-Prediction-Model")

In [50]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/samaTech-sys/Heart-Attack-Prediction-Model.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "SamaTech-sys"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "15c5a4aded0b84ca14020e0ad84ef46f6ab76f8f"

## Trial-runs for entity_config

In [51]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path 
    model_path: Path 
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

## Trial-runs for ConfigurationManager

In [52]:
#importing all project paths and modules necessary for project configurations 
from heartAttack.constants import *
from heartAttack.utils.common import read_yaml, create_directories

In [53]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        selected_schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.selected_schema = read_yaml(selected_schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ELASTICNET
        target_column = self.params.TARGET_COLUMN.name  # Accessing nested key

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=target_column,
            mlflow_uri="https://dagshub.com/samaTech-sys/Heart-Attack-Prediction-Model.mlflow"
        )
        
        return model_evaluation_config


## Trial-runs for component

In [54]:
import os 
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
import mlflow
import mlflow.sklearn
from heartAttack.utils.common import save_json

In [55]:
class ModelEvaluation: 
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred): 
        rmse = np.sqrt(mean_absolute_error(actual, pred))
        mse = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mse, r2

    def log_into_mlflow(self): 
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis =1)
        test_y = test_data[self.config.target_column]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mse, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            #saving metrics as local 
            scores = {"rmse": rmse, "mse": mse, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("r2", r2)
            
            #model registry does not work with filestore
            if tracking_url_type_store != "file":
                #Register the model 
                #There are other ways to use the model registry that depend on the use case
                #Please refer to the doc for more information 
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else: 
                mlflow.sklearn.log_model(model, "model")

## Trial-runs for the pipelines

In [56]:
#Update pipeline Step 
try: 
    config = ConfigurationManager()
    model_evaluation_config =config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-29 23:59:53,332: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-07-29 23:59:53,336: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-07-29 23:59:53,339: INFO: common: yaml_file: schema.yaml loaded successfully]
[2025-07-29 23:59:53,341: INFO: common: Created directory at: artifacts]
[2025-07-29 23:59:53,343: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-07-29 23:59:55,665: INFO: common: json file saved atartifacts\model_evaluation\metrics.json]


Successfully registered model 'ElasticnetModel'.
2025/07/30 00:00:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
